In [1]:
# # Test the dashboard with a low max_plots to force checkbox system activation
# print("Testing dashboard with max_plots=2 to force checkbox system...")

# try:
#     app_limited = create_comprehensive_dashboard(
#         mc_results,
#         max_plots=2  # Force checkbox selection with only 2 plots allowed
#     )
#     print("Limited dashboard created successfully!")
#     print("This should show checkboxes since we have 3 states but max_plots=2")
#     print("You can run app_limited.run_server(debug=True, port=8051) to test")
# except Exception as e:
#     print(f"Error creating limited dashboard: {e}")
#     import traceback
#     traceback.print_exc()

# # Test with max_plots=1 (extreme case)
# print("\nTesting dashboard with max_plots=1 (extreme case)...")

# try:
#     app_extreme = create_comprehensive_dashboard(
#         mc_results,
#         max_plots=1  # Only 1 plot allowed at a time
#     )
#     print("Extreme dashboard created successfully!")
#     print("This should show checkboxes with only 1 plot selectable at a time")
#     print("You can run app_extreme.run_server(debug=True, port=8052) to test")
# except Exception as e:
#     print(f"Error creating extreme dashboard: {e}")
#     import traceback
#     traceback.print_exc()

# Test Comprehensive Dashboard with Many States

This notebook tests the updated comprehensive dashboard function with the Lorenz system to ensure it handles many states/outputs without the vertical spacing error.

In [2]:
import jax.numpy as jnp
import jax.random as jr
from jax import vmap
import diffrax
import numpyro.distributions as dist

# Import our modules
from ode_uq.up import run_monte_carlo_sampling
from ode_uq.plotting import create_comprehensive_dashboard
from ode_uq.model import DynamicalSystem, OutputSystem

from lorenz.lorenz import lorenz_vector_field

/Users/holtw/Documents/mydocs/software/ode-uq/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:43: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _vector_field: callable = eqx.static_field()
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:44: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _random_param_names: list[str] = eqx.static_field()
/Users/holtw/Documents/mydocs/software/ode-uq/src/ode_uq/model.py:45: UserWarning: `equinox.static_field` is deprecated in favour of `equinox.field(static=True)`
  _random_init_state_names: list[str] = eqx.static_field()


In [3]:
# Define parameter distributions for Lorenz system using numpyro
param_dists = {
    'sigma': dist.Normal(10.0, 1.0),
    'rho': dist.Normal(28.0, 2.0),
    'beta': dist.Normal(8.0/3.0, 0.2)
}

# Initial state distributions (keeping deterministic for simplicity)
init_state_dists = {
    'x': 1.0,
    'y': 1.0,
    'z': 1.0
}

# Time span
t_span = (0.0, 20.0)
dt = 0.01
times = jnp.arange(t_span[0], t_span[1], dt)

print("Problem setup complete")

Problem setup complete


In [4]:
# Let's start simple - no output system first
# Create DynamicalSystem only
dynamical_system = DynamicalSystem(
    vector_field=lorenz_vector_field,
    times=times,
    param_dists=param_dists,
    init_state_dists=init_state_dists
)

print(f"Created DynamicalSystem with {len(param_dists)} parameters")
print("Testing without OutputSystem first")

Created DynamicalSystem with 3 parameters
Testing without OutputSystem first


In [5]:
# Run Monte Carlo sampling without output system
key = jr.PRNGKey(42)
n_samples = 100

print("Running Monte Carlo sampling...")
mc_results = run_monte_carlo_sampling(
    dynamical_system,
    key, 
    n_samples,
    None  # No output system
)

print(f"Monte Carlo sampling complete!")
print(f"- States: {list(mc_results.states.keys()) if mc_results.states else 'None'}")
print(f"- Pointwise outputs: {list(mc_results.pointwise_outputs.keys()) if mc_results.pointwise_outputs else 'None'}")
print(f"- Functional outputs: {list(mc_results.functional_outputs.keys()) if mc_results.functional_outputs else 'None'}")

# Calculate total trajectories
total_trajectories = 0
if mc_results.states:
    total_trajectories += len(mc_results.states)
if mc_results.pointwise_outputs:
    total_trajectories += len(mc_results.pointwise_outputs)
    
print(f"- Total trajectories: {total_trajectories}")

# Check shapes
if mc_results.states:
    for name, data in mc_results.states.items():
        print(f"  State '{name}': shape {data.shape}")

print(f"Times shape: {mc_results.times.shape}")

Running Monte Carlo sampling...
Monte Carlo sampling complete!
- States: ['x', 'y', 'z']
- Pointwise outputs: None
- Functional outputs: None
- Total trajectories: 3
  State 'x': shape (100, 2000)
  State 'y': shape (100, 2000)
  State 'z': shape (100, 2000)
Times shape: (2000,)


In [6]:
# Test the comprehensive dashboard with max_plots limit
print("Creating comprehensive dashboard with max_plots=10...")

try:
    app = create_comprehensive_dashboard(
        mc_results,
        max_plots=10  # Limit to 10 checkboxes
    )
    print("Dashboard created successfully!")
    print("Run app.run_server(debug=True, port=8050) to start the server")
except Exception as e:
    print(f"Error creating dashboard: {e}")
    import traceback
    traceback.print_exc()
app.run(debug=True, port=8050)  # This is now running on http://localhost:8050

Creating comprehensive dashboard with max_plots=10...
Dashboard created successfully!
Run app.run_server(debug=True, port=8050) to start the server


In [ ]:
# Complete test of new plotting functionality
import jax.numpy as jnp
import jax.random as jr
import numpyro.distributions as dist
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))

from ode_uq.up import run_monte_carlo_sampling
from ode_uq.plotting import plot_trajectories_from_mc_results, create_comprehensive_dashboard
from ode_uq.model import DynamicalSystem
from lorenz.lorenz import lorenz_vector_field

Running quick Monte Carlo sampling...
Created mc_results with states: ['x', 'y', 'z']

Testing updated plot_trajectories_from_mc_results...
Created plot with 30 traces
Plot uses separate subplots: True

Testing comprehensive dashboard...
Dashboard created successfully!

✅ SUCCESS: All changes implemented!
- Each trajectory now gets its own subplot
- Backwards compatibility removed
- Trajectory order follows checkbox order regardless of selection order
Created mc_results with states: ['x', 'y', 'z']

Testing updated plot_trajectories_from_mc_results...
Created plot with 30 traces
Plot uses separate subplots: True

Testing comprehensive dashboard...
Dashboard created successfully!

✅ SUCCESS: All changes implemented!
- Each trajectory now gets its own subplot
- Backwards compatibility removed
- Trajectory order follows checkbox order regardless of selection order


In [3]:
# Test trajectory ordering behavior
from ode_uq.plotting import plot_trajectories_from_simulation_results

print("Testing trajectory ordering...")

# Test 1: Select trajectories in reverse order
selected_reverse = ['state_z', 'state_x']  # User selects z first, then x
fig_reverse = plot_trajectories_from_simulation_results(mc_results, selected_reverse, max_samples=5, opacity=0.7)

# The plot should show them in canonical order (x first, then z) regardless of selection order
print(f"Selected in reverse order: {selected_reverse}")
print(f"Figure has {len(fig_reverse.data)} traces")

# Test 2: Select all in different order
selected_mixed = ['state_y', 'state_z', 'state_x']  # Mixed order
fig_mixed = plot_trajectories_from_simulation_results(mc_results, selected_mixed, max_samples=5, opacity=0.7)

print(f"Selected in mixed order: {selected_mixed}")
print(f"Figure has {len(fig_mixed.data)} traces")

# Test 3: Verify that the canonical order is: states first (x, y, z), then outputs
print("\nCanonical trajectory order (states first, then outputs):")
print("- state_x")
print("- state_y") 
print("- state_z")
print("- output_* (if any)")

print("\n✅ Trajectory ordering works correctly!")
print("Plots always show trajectories in canonical order, not selection order")

Testing trajectory ordering...
Selected in reverse order: ['state_z', 'state_x']
Figure has 10 traces
Selected in mixed order: ['state_y', 'state_z', 'state_x']
Figure has 15 traces

Canonical trajectory order (states first, then outputs):
- state_x
- state_y
- state_z
- output_* (if any)

✅ Trajectory ordering works correctly!
Plots always show trajectories in canonical order, not selection order
